In [2]:
import tensorflow as tf
import numpy as np

In [3]:
samples = 200

In [4]:
x = np.random.randn(samples,128,32)

In [5]:
y = np.random.choice([0,1],size=samples)

In [6]:
data = list(zip(x,y))

In [7]:
class DataSequenceRaw(tf.keras.utils.Sequence):

    def __init__(self,data,batch_size):
        self.data_size = len(data)
        audios,labels = zip(*data)
        self.X = np.array(audios)
        self.Y = np.array(labels)
        self.batch_size = batch_size

        
    def __len__(self):
        return int(np.ceil(self.data_size / self.batch_size))

    def __getitem__(self,idx):
        s = idx * self.batch_size
        e = (idx + 1) * self.batch_size
        X = self.X[s:e]
        Y = self.Y[s:e]
        return X,Y


In [8]:
def _get_train_val_size(total_examples,train_percent=70):
    
    train_ratio = round(train_percent)/100
    remaining_percent = 100 - train_percent
    test_percent = round(remaining_percent/3)*2
    val_percent = remaining_percent - test_percent
    val_ratio = val_percent/100
    train_size = round(total_examples * train_ratio)
    val_size = round(total_examples * val_ratio)
    return train_size,val_size

def get_data_raw(data,train_percent=70,batch_size=32):

    train_size,val_size = _get_train_val_size(len(data),train_percent)

    train_examples = data[:train_size]
    val_examples = data[train_size:train_size+val_size]
    test_examples = data[train_size+val_size:]

    train = DataSequenceRaw(train_examples,batch_size=batch_size)
    test = DataSequenceRaw(test_examples,batch_size=1)
    val = DataSequenceRaw(val_examples,batch_size=batch_size)

    return train,test,val

train,test,val = get_data_raw(data,train_percent=80)

In [9]:
shape = train[0][0][0].shape
input_shape = [*shape,1]

In [10]:
def get_model(
        input_shape,
        output_neurons=1,
        output_activation='sigmoid',
        loss=tf.keras.losses.binary_crossentropy,
        lr=0.0001
):
    _input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(512,kernel_size=3,padding='valid',activation='relu')(_input)
    x = tf.keras.layers.Conv2D(256,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(64,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dense(10,activation='relu')(x)
    outputs = tf.keras.layers.Dense(output_neurons,activation=output_activation,kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
    model = tf.keras.Model(inputs=_input,outputs=outputs)

    model.compile(
        loss=loss,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'],
    )

    return model

model = get_model(
        input_shape=input_shape,
        lr=0.001
)



In [12]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=5,mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=1,mode='max',restore_best_weights=True,start_from_epoch=10)
# with tf.device('/gpu'):
history = model.fit(train,epochs=3,validation_data=val,verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/3
5/5 [==============================] - 9s 2s/step - loss: 0.7083 - accuracy: 0.5813 - val_loss: 0.7023 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 2/3
5/5 [==============================] - 10s 2s/step - loss: 0.6988 - accuracy: 0.5813 - val_loss: 0.7001 - val_accuracy: 0.5833 - lr: 0.0010
Epoch 3/3
5/5 [==============================] - 10s 2s/step - loss: 0.7008 - accuracy: 0.5813 - val_loss: 0.6993 - val_accuracy: 0.5833 - lr: 0.0010


In [13]:
model.save(r'C:\Users\muhammaddaniyal2\Desktop\keyword_spotting\models\model.keras')